In [1]:
import utils
import models
import model_utils
import torch.nn as nn
import torch.optim as optim

In [2]:
# dl = model_utils.DataLoader("mel_stft")
import numpy as np
import torch
data = torch.from_numpy(np.load("data/features/mel_stft/train_mel_stft_0.npy")).float()

In [3]:
test_cnn = models.MelCNNFreq()
test_cnn(data

tensor([[ 0.0945, -0.2247, -0.1333, -0.0688,  0.1374, -0.1486, -0.0144, -0.1298],
        [ 0.0663, -0.2006, -0.1298, -0.1204,  0.0901, -0.0985, -0.0889, -0.1466],
        [-0.0114, -0.1573, -0.0850, -0.1083,  0.0925, -0.0868, -0.0255, -0.0307],
        [-0.0587, -0.1578, -0.0338, -0.0700,  0.0480, -0.0512,  0.0411,  0.0542]],
       grad_fn=<AddmmBackward>)